# ASL Classification with a CNN

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Get reproducible results
random_state = 46
tf.random.set_seed(random_state)

## GPU configuration
If you have a GPU, enable experimental memory growth.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

## File Paths

In [ ]:
train_dir = "asl_alphabet_train/asl_alphabet_train"
test_dir = "asl_alphabet_test/asl_alphabet_test"

## Train model

In [ ]:
def fit_model(model):
    
    print("Defining train data generator...")
    datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=False,
        featurewise_std_normalization=False,
        rotation_range=30,
        validation_split=0.30)
    
    train_generator = datagen.flow_from_directory(
        train_dir,
        subset='training',
        target_size=(200, 200),
        batch_size=128,
        class_mode='sparse'
    )
    
    val_generator = datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=(200, 200),
        batch_size=128,
        class_mode='sparse'
    )
    
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    model.summary()
    
    print('Fitting model...')
    history = model.fit(train_generator, 
        steps_per_epoch=100, 
        epochs=20, 
        verbose=1, 
        validation_data=val_generator, 
        validation_steps=50,
        use_multiprocessing=True,
        workers=16)
    
    print('Evaluating model...')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    
    del model
    K.clear_session()
    
def fit_with_base_model(base_model, trainable = False):
    base_model.trainable = trainable
    fit_model(tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(29)
    ]))
    

# Base models

## InceptionResNetV2
This model uses InceptionV3 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet'))

## InceptionV3
This model uses InceptionV3 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.InceptionV3(include_top=False, weights='imagenet'))


## MobileNet
This model uses MobileNet as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.MobileNet(include_top=False, weights='imagenet'))


## MobileNetV2
This model uses MobileNetV2 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet'))

## ResNet50
This model uses ResNet50 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.ResNet50(include_top=False, weights='imagenet'))

## ResNet50V2
This model uses ResNet50 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.ResNet50(include_top=False, weights='imagenet'))

## VGG16
This model uses VGG16 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.VGG16(include_top=False, weights='imagenet'))

## VGG19
This model uses VGG16 as a base model initialized with the ImageNet weights.  

In [ ]:
fit_with_base_model(tf.keras.applications.VGG19(include_top=False, weights='imagenet'))

### In case, a kernel is interrupted, to free up GPU memory, run:

In [ ]:
try: 
    del model
except NameError: model = None

K.clear_session()